In [1]:
import pandas as pd

In [2]:
file_path = 'recommend_trx_simple/train.csv'
data = pd.read_csv(file_path)
file_path = 'recommend_trx_simple/test.feature.csv'
data_test = pd.read_csv(file_path)

data

,reqId,f0_0,f0_1,f0_2,f0_3,f0_4,f0_5,f0_6,f0_7,f0_8,...,f9_11,f9_12,f9_13,f9_14,f9_15,f9_16,f9_17,f9_18,f9_19,label
0,2046312,13.1333,-5.5349,9.9136,10.1509,9.6477,-13.4910,4.0839,17.0913,3.2179,...,5.9957,2.2689,0.3879,21.6136,-1.6631,-1.5878,9.5645,14.3934,2.9574,0
1,2199280,8.6115,1.7219,9.9564,8.2634,10.8537,-12.3207,6.3826,14.4280,-1.4802,...,7.7096,-0.3909,0.0473,21.7868,0.3931,9.5753,10.2083,15.8513,-14.8251,0
2,2187412,8.1161,-6.9239,9.4631,3.2508,7.4479,-2.8949,3.8622,18.9906,0.7808,...,8.2638,2.5743,-2.3667,23.1186,-0.5729,-0.2207,8.8768,15.5500,-2.3945,0
3,2092894,9.5935,-8.1052,11.3833,3.3209,12.8809,1.1193,5.4555,12.7347,5.5715,...,3.2582,1.3695,-2.8866,17.2128,1.0166,-4.9866,8.9236,20.2622,-14.2090,0
4,2157648,15.7832,-0.1155,12.5025,5.5413,8.7752,-6.5656,4.6026,15.5878,1.1430,...,7.2594,1.3949,2.4469,18.9961,-2.9151,-0.8292,10.3700,16.8027,-7.0793,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179995,2199981,12.8140,0.6386,14.1657,7.1044,8.9365,-0.3274,6.5949,14.6078,-1.0373,...,4.9010,2.2513,0.7308,14.7155,1.1464,5.5158,8.6519,16.0341,7.3809,1
179996,2199980,7.9618,6.2271,13.3326,7.5345,12.5645,1.1205,5.4488,22.1532,4.5604,...,7.6274,4.7426,4.6682,21.1164,1.1648,1.6012,8.2107,14.5403,-8.7837,0
179997,2199978,12.2015,-0.8787,16.8069,9.1073,10.3219,7.0208,6.2587,19.8344,4.9897,...,5.5487,2.0673,-5.6931,16.5850,-1.9920,3.1963,7.5484,13.1150,6.7866,0
179998,2199976,7.9663,-2.8485,9.0919,7.3298,9.6690,-16.7872,4.5094,12.4351,-0.0113,...,4.3876,3.7257,7.7038,14.7384,0.1561,1.5794,8.4627,14.3604,-1.6688,1


In [16]:
data['f0_0'].dtypes

dtype('float64')

In [4]:
data['label'].value_counts()

label
0    161899
1     18101
Name: count, dtype: int64

In [5]:
# 分离特征和目标列
X = data.drop(columns=['label', 'reqId'])  
y = data['label']
X_data_test = data_test.drop(columns=['reqId'])

In [6]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [7]:
print(f"Original data distribution: {Counter(y)}")
# 初始化 SMOTE
smote = SMOTE(random_state=42)

features_smote, label_smote = smote.fit_resample(X, y)

print(f"Data distribution after applying SMOTE: {Counter(label_smote)}")

Original data distribution: Counter({0: 161899, 1: 18101})
Data distribution after applying SMOTE: Counter({0: 161899, 1: 161899})


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features_smote, label_smote, test_size=0.2, random_state=42)

In [10]:
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, roc_auc_score

In [11]:

# 创建和训练 XGBoost 分类器
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# 评估模型
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy:.2f}')
print(f'ROC AUC: {roc_auc:.2f}')


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [00:51:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.90
ROC AUC: 0.96


In [12]:
result = pd.DataFrame(y_pred)

In [13]:
result.value_counts()

0
0    32904
1    31856
Name: count, dtype: int64

In [19]:
# 在测试集上进行预测
y_pred = model.predict(X_data_test)
y_pred_proba = model.predict_proba(X_data_test)[:, 1]
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [20]:
result = pd.DataFrame(y_pred)

In [21]:
result.rename(columns={0: 'label'},inplace=True)

In [22]:
result

,label
0,0
1,0
2,0
3,0
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [23]:
result.value_counts()

label
0        17536
1         2464
Name: count, dtype: int64

In [18]:

from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 基础模型
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# 参数分布
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    # 'gamma': np.linspace(0, 0.4, 5),  # 连续分布的参数
    'reg_alpha': [0, 0.1, 0.5],  # L1 正则化
    # 'reg_lambda': [1, 1.5, 2]  # L2 正则化
}

# 随机搜索
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=30,  
    scoring='roc_auc',
    cv=3,
    verbose=2,
    random_state=42,
)

# 训练随机搜索模型
random_search.fit(X_train, y_train)

# 打印最佳参数
print("Best parameters found: ", random_search.best_params_)
print("Best AUC score: ", random_search.best_score_)


Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [01:07:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.6, learning_rate=0.05, max_depth=9, min_child_weight=7, n_estimators=300, reg_alpha=0.1, subsample=1.0; total time=  17.0s


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [01:07:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.6, learning_rate=0.05, max_depth=9, min_child_weight=7, n_estimators=300, reg_alpha=0.1, subsample=1.0; total time=  15.8s


...

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, min_child_weight=7, n_estimators=100, reg_alpha=0.5, subsample=1.0; total time=   6.3s


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [01:16:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=9, min_child_weight=7, n_estimators=100, reg_alpha=0.5, subsample=1.0; total time=   5.9s


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [01:16:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best parameters found:  {'subsample': 1.0, 'reg_alpha': 0, 'n_estimators': 200, 'min_child_weight': 5, 'max_depth': 9, 'learning_rate': 0.2, 'colsample_bytree': 0.6}
Best AUC score:  0.9828145823907389


In [19]:
# 提取最佳模型
best_model = random_search.best_estimator_  
best_model.fit(X_train, y_train)
# 预测类别
y_pred = best_model.predict(X_data_test)


C:\Users\yjq03\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [01:19:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [20]:
result = pd.DataFrame(y_pred)

In [21]:
result.value_counts()

0
0    18574
1     1426
Name: count, dtype: int64

In [23]:
result.rename(columns={0: 'label'},inplace=True)

In [24]:
result

,label
0,0
1,0
2,0
3,0
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [25]:
result.to_csv('recommend_trx_simple.csv', index=False)